In [1]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

!pip install colab-xterm -qqq
!pip install langchain -qqq
!pip install langchain_community -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [2]:
!pip install langchain_neo4j -qqq
!pip install langchain_openai  -qqq
!pip install langchain_google_genai  -qqq
!pip install langchain_huggingface -qqq
!pip install langchain_experimental -qqq
!pip install jq -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 22.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [3]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader,JSONLoader
from langchain_neo4j import Neo4jVector,Neo4jGraph,GraphCypherQAChain
from langchain_openai import OpenAIEmbeddings

In [ ]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import json
from pathlib import Path
from langchain.document_loaders import JSONLoader  # Make sure this is installed

# Path to your JSON file
file_path = '/content/processed_with_metadata.json'

# JSON root is a list of dictionaries — so use .[] to iterate each record
jq_schema = '.[]'

# Field that contains main document text
content_key = 'text'

# Function to extract structured metadata from each record
def metadata_func(record: dict, metadata: dict) -> dict:
    headings = record.get('headings', [])
    media = record.get('media_links', {})
    anchor_links = record.get('anchors', [])

    metadata['url'] = record.get('url')
    metadata['page_name'] = record.get('page')
    metadata['nav_path'] = record.get('nav_path', [])
    metadata['title'] = headings[1] if len(headings) > 1 else "N/A"
    metadata['categories'] = list(set(h for h in headings if h.strip()))

    metadata['pdf_count'] = len(media.get('pdfs', []))
    metadata['image_count'] = len(media.get('images', []))
    metadata['video_count'] = len(media.get('videos', []))
    metadata['total_anchors'] = len(anchor_links)
    metadata['linked_domains'] = list({link.split("/")[2] for link in anchor_links if link.startswith("http")})

    return metadata

# Load the documents
loader = JSONLoader(
    file_path=file_path,
    jq_schema=jq_schema,
    # content_key=content_key,
    # metadata_func=metadata_func,
    text_content=False
)

documents = loader.load()




In [ ]:
# loader = TextLoader("sample.txt")
# loader = TextLoader("data.json", encoding="utf-8")
# documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

139

In [ ]:
# docs[110]

In [4]:
uri= "neo4j+s://691cf4c8.databases.neo4j.io"
username= "neo4j"
password= "V7oAh2uCKq7ivrzqZwRIk5FPRLi0SGUVnycxH1Xjwmo"

In [ ]:
db = Neo4jVector.from_documents(
    docs, embeddings, url=uri, username=username, password=password,
    search_type="hybrid"
)

In [ ]:
query = "name the satelitte project in sriharikotta"
docs_with_score = db.similarity_search_with_score(query, k=2)

In [ ]:
docs_with_score

[(Document(metadata={'linked_domains': ['www.mosdac.gov.in'], 'seq_num': 10, 'image_count': 11, 'url': 'https://www.mosdac.gov.in/about-us', 'title': 'You are here', 'source': '/Users/sec22ad081/Desktop/kg/data.json', 'pdf_count': 1, 'categories': ['You are here', 'Search', 'Secondary menu', 'About Us', 'Follow Us'], 'nav_path': ['Homepage', 'www.mosdac.gov.in', 'imageshow', 'terms-conditions', 'copyright-policy'], 'total_anchors': 60, 'page_name': 'about-us', 'video_count': 0}, page_content='Secondary menu\nYou are here\nAbout Us\nSearch\nFollow Us\nSkip to main Content\nSpace Applications Centre (SAC) is an ISRO Centre located at Ahmedabad, dealing with a wide variety of themes from satellite payload development, operational data reception and processing to societal applications. SAC is responsible for the development, realization and qualification of communication, navigation, earth observation and planetary payloads and related data processing and ground systems in the areas of com

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [5]:
from langchain.chat_models import init_chat_model

In [6]:
llm = init_chat_model(model ='mistral',model_provider='ollama')
# llm = init_chat_model(model = "llama-3.3-70b-versatile",model_provider='groq')

In [ ]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = await llm_transformer.aconvert_to_graph_documents(docs[:2])

In [ ]:
print(f"Nodes:{graph_documents[1].nodes}")
print(f"Relationships:{graph_documents[1].relationships}")

Nodes:[Node(id='Meteorological & Oceanographic Satellite Data Archival Centre', type='Organization', properties={}), Node(id='Indian Space Research Organisation', type='Organization', properties={}), Node(id='Govt. Of India', type='Organization', properties={})]
Relationships:[Relationship(source=Node(id='Meteorological & Oceanographic Satellite Data Archival Centre', type='Organization', properties={}), target=Node(id='Indian Space Research Organisation', type='Organization', properties={}), type='MAINTAINED_BY', properties={}), Relationship(source=Node(id='Indian Space Research Organisation', type='Organization', properties={}), target=Node(id='Govt. Of India', type='Organization', properties={}), type='GOVERNED_BY', properties={})]


In [ ]:
graph_documents[1].nodes

[Node(id='Meteorological & Oceanographic Satellite Data Archival Centre', type='Organization', properties={}),
 Node(id='Indian Space Research Organisation', type='Organization', properties={}),
 Node(id='Govt. Of India', type='Organization', properties={})]

In [8]:
graph = Neo4jGraph(url=uri, username=username, password=password)

In [ ]:
graph.add_graph_documents(graph_documents)

In [ ]:
!pip install yfiles_jupyter_graphs -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.3 MB/s eta 0:00:00


In [ ]:
import os
from neo4j import GraphDatabase
from IPython.display import display
from yfiles_jupyter_graphs import GraphWidget  # Replace with actual widget library you're using

def showGraph(cypher: str = "MATCH (n)-[r]->(m) RETURN n, r, m"):
    # Create Neo4j driver using environment variables
    driver = GraphDatabase.driver(
        uri,
        auth=(username, password)
    )

    # Create a session
    with driver.session() as session:
        # Run the Cypher query
        result = session.run(cypher)

        # Create graph widget
        widget = GraphWidget(graph=result.graph())
        widget.node_label_mapping = 'id'  # Adjust based on your node properties

        # Display the widget
        display(widget)

    return widget

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
docs[0].page_content

'{"url": "https://www.mosdac.gov.in", "title": "Meteorological & Oceanographic Satellite Data Archival Centre", "text": "Skip to main Content 3-hourly Weather forecast. Cold wave prediction over India (WRF Model) Sat. Based Cyclone Obser. and Realtime Pred. over IO. Heat Wave Prediction over India (WRF Model) Heavy Rain (>5mm/hr) forecast using NWP model. Lightning Forecast Monsoon Prediction 2024 Sea State forecast in terms of Wave Height, Wave period etc. 3 Days Solar and Wind Forecast for every 15 minutes Cloud Burst Nowcast for 6 hours\\u00a0over Western Himalayan region. Satellite based nowcast for Heavy rain events for 6 hours. AWS Time Series Heavy Rain Events Let\'s Interactively Visualise the Earth (LIVE) Alerts for Indian States Alerts for\\u00a0South East Asian Countries Solar Energy Rainfall Air Quality Soil Wetness Index Sat. Based Cyclone Obser. and Realtime Pred. over IO. ALTIMETER LCS-CORES AND STRETCHING DIRECTIONS Ocean Subsurface Fields RIP Current Forecast for All-I

In [ ]:
# ollm = init_chat_model(model='llama3.1:latest',model_provider='ollama')

In [9]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
    allow_dangerous_requests=True
)

In [12]:
chain.run("where the data processing system sends data?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (system:System)-[:SENDS_DATA_TO]->(receiver:System)
RETURN receiver
Full Context:
[{'receiver': {'id': 'Data Processing System'}}, {'receiver': {'id': 'Product Generation And Dissemination System'}}]

> Finished chain.


' The Data Processing System sends data to the Product Generation and Dissemination System.'

In [ ]:
#it is just initial RnD of this project
#and we successfully completed the ground work and now it is devoloping efficiently